In [ ]:
import os
import pandas as pd
import Blobconfig

# Access blob_service_client and container_client from Blobconfig
blob_service_client = Blobconfig.blob_service_client
container_client = Blobconfig.container_client

csv_file_path = "HLP_automated_extraction.csv"

zip_folder_path = os.path.join(os.path.dirname(csv_file_path), "zip")
os.makedirs(zip_folder_path, exist_ok=True)

existing_files = os.listdir(zip_folder_path)
for file in existing_files:
    file_path = os.path.join(zip_folder_path, file)
    os.remove(file_path)

%run Trigger_All_Full_Extract.py

downloads_folder_path = os.path.join(os.path.expanduser("~"), "Downloads")
zip_files = [f for f in os.listdir(downloads_folder_path) if f.endswith(".zip")]

for zip_file in zip_files:
    zip_file_path = os.path.join(downloads_folder_path, zip_file)
    new_zip_file_path = os.path.join(zip_folder_path, zip_file)

    ten_minutes_ago = pd.Timestamp.now() - pd.Timedelta(minutes=10)
    creation_time = pd.Timestamp.fromtimestamp(os.path.getctime(zip_file_path))

    if creation_time > ten_minutes_ago:
        os.rename(zip_file_path, new_zip_file_path)
        print(f"{zip_file_path} => {new_zip_file_path}")

print("Toutes les archives ZIP ont été déplacées vers le dossier 'zip' sur le bureau.")
print(f"Creation Time is {creation_time}\n\n\n")

%run "Boe BI Automated Extraction Daily/Extract-Xlsx_From_Outlook_For_Power_Automate_Trigger_Blob.py"
%run Unzip_Move_Rename.py
%run remove_date.py
%run move_data.py
%run Delete_zip.py
